In [7]:
import os
import os.path as ospath
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil
import glob
from tqdm.notebook import tqdm
from scipy import interpolate
from sklearn.neighbors import NearestNeighbors as NN
from scipy.signal import find_peaks

from picasso.picasso import io

import warnings



In [16]:
blue = "#4C72B0"

def load_ring_data_df(dirname, filename):
    file = ospath.join(dirname, filename)

    try: 
        df = pd.read_pickle(file)
    except FileNotFoundError:
        print("No results of previously analyzed datasets were detected.")
        return None
    else: 
        print("Results of previously analyzed datasets were detected.")
        return df
    
def load_data(path):

    try:
        locs, info = io.load_locs(path)
    except io.NoMetadataFileError:
        return None, None, None
    
    try:
        pixelsize = info[1]["Pixelsize"]
    except:  
        print("No pixelsize found in yaml file. Default 130 nm used.")

        
    return locs, info, pixelsize


def nearest_neighbor(
    x1, x2, 
    y1, y2, 
    z1, z2,
    nn_count, 
    same_channel, 
):
    """
    For each point in dataset 2, search the nearest neighbor in
    dataset 1.
    """

    
    
    if z1 is not None: # 3D
        input1 = np.stack((x1, y1, z1)).T
        input2 = np.stack((x2, y2, z2)).T
    else: # 2D
        input1 = np.stack((x1, y1)).T
        input2 = np.stack((x2, y2)).T
        
    # if there are less datapoints in the dataset 
    # in which we are searching the nearest neighbors
    # than the number of higher nearest neigbhors to be calculated

    if same_channel:
        if nn_count >= len(input1):
            nn_count = len(input1)-1
    else:
        if nn_count > len(input1):
            nn_count = len(input1)
        
        
    if same_channel:
        model = NN(n_neighbors=nn_count+1)
    else:
        model = NN(n_neighbors=nn_count)
    model.fit(input1)
    try:
        nn, idx = model.kneighbors(input2)
    except ValueError:
        print('input 1', len(input1))
        print('input 1', input1)
        print('input 2', len(input2))
        print('input 2', input2)
        sys.exit()
        
    if same_channel:
        nn = nn[:, 1:] # ignore the zero distance
        idx = idx[:, 1:]
    
    # idx has the shape len(input2)x1 (where len(input2) = len(nn))
    # Indices of the nearest points in the population matrix.
    # It could have such a shape [[1] [0] [2] [2] ... ]
    # The value at index i is the index of the nearest neighbor 
    # in dataset 1 of the point in dataset 2 at index i.
    return nn, idx

In [33]:
path = r'W:\users\reinhardt\z.software\Git\spor-PAINT\dev_sr\spor-paint\SepF\picked_rings_SepF_working'

dR = 100
protein = 'SepF'

folder_suffix = ''

df_ring_data = load_ring_data_df(path, "ring_data_filter.pkl")

Results of previously analyzed datasets were detected.


In [34]:
## unfolding
warnings.filterwarnings("ignore", category=RuntimeWarning) 

orientation = 'xy' # use yz, but also xy files contain info for filtering

spline_folder = os.path.join(path, 'analysis', 'unfold'+folder_suffix)
spline_folder_exclude = os.path.join(path, 'analysis', 'unfold'+folder_suffix, 'excluded_filter')

if not os.path.isdir(spline_folder):
    os.makedirs(spline_folder)
if not os.path.isdir(spline_folder_exclude):
    os.makedirs(spline_folder_exclude)



df_ring_data['filename_ring'] =  'fov_' + df_ring_data['fov_id'].astype(str) +'_' + df_ring_data['cell_type'].astype(str) +'_pick_' + df_ring_data['group'].astype(str) +'_ring_' + '0' +'_rot_' + orientation + '_update.hdf5'
for index, ring in tqdm(df_ring_data.iterrows(), desc="Processing rings", total = df_ring_data.shape[0]):
    plt.close('all')
    
    Radius = ring['radius']
    radius_min = Radius - dR
    radius_max = Radius + dR
    
    
    locs, info, pixelsize = load_data(os.path.join(path, 'analysis', 'ring_locs', ring['filename_ring'] ))
    #print(ring['filename_ring'])

    if ring['filter_passed'] == 'Yes':
        
        
    
        df_locs = pd.DataFrame.from_records(locs)

        # filter
        df_locs = df_locs[(df_locs['radius']>radius_min) & (df_locs['radius']<radius_max)]

        df_locs = df_locs.sort_values(by='angle')
        #print('angle', df_locs['angle'].min(), df_locs['angle'].max(), )



        theta = df_locs['angle']/180*np.pi
        R = df_locs['radius']

        x = R * np.cos(theta)
        y = R * np.sin(theta)




        ### Graph

        linewidth = 2
        fontsize_main_title = 25
        fontsize_title = 20
        fontsize_text = 18


        fig = plt.figure(figsize=(15, 20)) #, constrained_layout=True
        gs = fig.add_gridspec(3,1, height_ratios=[1.5,1,1], width_ratios=[1])
        fig.suptitle(("FOV {}, {}, Pick {} - Unfolding\n"
                      "{}: {}").format(ring['fov_id'], ring['cell_type'], ring['group'], protein, ring['filename_ring']),
                      fontsize=fontsize_main_title,
                      ha="center")








        ## spline fit
        def interpol3D(num_true_pts, s, x_array, y_array, z_array, per = 0):

            tck, u = interpolate.splprep([x_array, y_array, z_array], s=s, per = per) # adding per = 1 fixed it
            # default k = 3 -> cubic spline

            x_knots, y_knots, z_knots = interpolate.splev(tck[0], tck)
            u_fine = np.linspace(0, 1, num_true_pts)
            x_fine, y_fine, z_fine = interpolate.splev(u_fine, tck)

            return x_fine, y_fine, z_fine



        # interpolation parameters ###

        m = len(df_locs)
        #s = (m-np.sqrt(2*m)) * 50**2
        s = m * 50**2 / 2
        num_true_pts = 3601    


        x_fine, y_fine, z_fine, = interpol3D(num_true_pts, s, x, y, np.zeros(len(x)), per = 1)


        # Illustrate spline fit: 
        ax1 = fig.add_subplot(gs[0,0])

        # plot locs
        ax1.scatter(x, y, label= 'Localizations', alpha = 0.3, color = blue);





        # plot fit
        #ax1.plot(x_fit, y_fit, label="Fitted circle", lw=linewidth, c=red)

        ax1.set_xlabel('x (nm)', fontsize=fontsize_text)
        ax1.set_ylabel('y (nm)', fontsize=fontsize_text)
        ax1.axes.set_aspect('equal')
        ax1.set_title("Spline fit",loc="left",fontsize=fontsize_title)
        ax1.tick_params(axis='x', labelsize=fontsize_text)
        ax1.tick_params(axis='y', labelsize=fontsize_text)
        ax1.plot(x_fine, y_fine, color = 'lime', label = 'Spline fit', linewidth = linewidth)
        ax1.legend(fontsize=fontsize_text)









        ### unfold

        # calculate distance of each datapoint to the spline

        #The spline consists of num_true_pts coordinates. 
        #-> Calculate the distance from a given datapoint to all spline coordinates.

        # search the nearest neighbor 
        nnd, idx = nearest_neighbor(
                    x_fine, x, 
                    y_fine, y, 
                    z1 = None, z2 = None,
                    nn_count = 1,
                    same_channel = False)


        # first and last point in spline are identical
        idx[idx==idx.max()] = idx.min()


        def spline_dist(xs, ys, zs):
            # distance from one point to the next
            dist = np.sqrt((xs[1:]-xs[:-1])**2 + (ys[1:]-ys[:-1])**2 + (zs[1:]-zs[:-1])**2)
            dist_cum = np.cumsum(dist)

            return dist_cum

        dist_cum = spline_dist(x_fine, y_fine, z_fine)



        x_unfold = np.squeeze(dist_cum[idx])
        y_unfold = np.squeeze(nnd)


        # nnd is always positive.
        # Is the datapoint inside of the spline or outside?
        # Calculate if the datapoint or its closest point on the spline are closer to the center
        sign_bool = np.less(x**2 + y**2,np.squeeze(x_fine[idx])**2+np.squeeze(y_fine[idx])**2)
        sign = sign_bool.copy()
        sign[~sign_bool] = +1
        sign[sign_bool] = -1


        y_unfold = sign * y_unfold



        ax2 = fig.add_subplot(gs[1,0])
        ax2.scatter(x_unfold, y_unfold, label= 'Unfolded localizations', alpha = 0.3, color = blue);
        ax2.plot(dist_cum, np.full(len(dist_cum),0), color = 'lime', linewidth = linewidth)
        ax2.legend(fontsize=fontsize_text)
        ax2.set_xlabel('x (nm, distance along spline)', fontsize=fontsize_text)
        ax2.set_ylabel('y (nm, distance to spline)', fontsize=fontsize_text)
        ax2.axes.set_aspect('equal')
        ax2.set_title("Unfolded localizations along spline fit",loc="left",fontsize=fontsize_title)
        ax2.tick_params(axis='x', labelsize=fontsize_text)
        ax2.tick_params(axis='y', labelsize=fontsize_text)


        # histogram
        ax3 = fig.add_subplot(gs[2,0])

        step_size = 15 # nm
        bins = np.arange(x_unfold.min(),x_unfold.max()+step_size/2, step_size)
        centers = (bins[:-1] + bins[1:]) / 2
        binwidth = bins[1]-bins[0]

        n, bins = np.histogram(x_unfold, bins=bins)


        ax3.bar(centers, n, width=binwidth, color=blue, alpha = 0.5, label = 'histogram')


        # search peaks
        peak_inx, peak_properties = find_peaks(n, height=n.mean())
        #peak_inx_1, peak_properties_1 = find_peaks(n1, height=n1.max()/4)
        #peak_inx_1, peak_properties_1 = find_peaks(n1, prominence = (0.2,1))
        ax3.scatter(centers[peak_inx], n[peak_inx], marker = "X", color = blue, s = 100, label = 'peaks')
        ax3.annotate('$N_{peaks}$ = %d' % len(peak_inx), (0.7,0.9), xycoords = 'axes fraction', fontsize = fontsize_text)    


        ax3.set_title("Line profile along spline fit",loc="left",fontsize=fontsize_title)


        ax3.set_xlabel("x (nm)", fontsize=fontsize_text)
        ax3.set_ylabel("Counts", fontsize=fontsize_text) 
        ax3.set_xlim(ax2.get_xlim())
        #ax3.set_xticklabels(fontsize=fontsize_text)
        ax3.tick_params(axis='x', labelsize=fontsize_text)
        ax3.tick_params(axis='y', labelsize=fontsize_text)
        ax3.legend(fontsize=fontsize_text)



        ## Filter out uggly rings:
        # criteria
        # - The length of the spline curve is much larger or smaller than the circle circumference
        # - To filter out rings that are incomplete (most likely because the imaging plane was off)
        #   I will use an approach similar to the filtering by the number of segments that we used for single channel data.
        #   For simplicity I will here define segments along the spline curve (aka segments in the histogram) and 
        #   discard a ring if too many sigments are empty


        # circumference criterion
        circumference = 2 * Radius * np.pi
        spline_length = dist_cum[-1]

        exclude = False

        if spline_length >= (1.5 * circumference):
            exclude = True
        if spline_length <= (0.5 * circumference):
            exclude = True

        #print('spline_length:', spline_length)
        #print('circumference:', circumference)
        #print('spline_length/circumference:', spline_length/circumference)


        # segment criterion

        # Sum the histogram bins of channel 1 and 2
        N_segments = 12
        perc_locs_per_filled_segment_min = 1/N_segments/4 # a segment counts as filled if it contains this fraction of all localizations
        perc_segments_filled_min = 0.6 # this fraction of segments has to be filled to accept the ring

        length_segment = np.ceil(len(n) / N_segments).astype(int) # number of bins in histogram per segment
        locs_per_segment = np.add.reduceat(n, np.arange(0, len(n), length_segment))

        perc_locs_per_segment = locs_per_segment / n.sum()
        #print('perc_locs_per_segment:', perc_locs_per_segment)

        perc_locs_per_segments_filled = perc_locs_per_segment[perc_locs_per_segment > perc_locs_per_filled_segment_min]
        perc_segments_filled = len(perc_locs_per_segments_filled)/N_segments
        #print('perc_segments_filled:', perc_segments_filled)
        if perc_segments_filled < perc_segments_filled_min:
            exclude = True

        #plt.show()

        # save
        fig.tight_layout(rect=[0, 0.03, 1, 0.97])
        filename_save = ring['filename_ring'].replace('.hdf5', '_spline.png')

        df_ring_data.loc[index,'2piradius_merge'] = circumference
        df_ring_data.loc[index,'spline_length'] = spline_length
        df_ring_data.loc[index,'N_peaks'] = len(peak_inx)


        if not exclude:
            df_ring_data.loc[index,'spline_filter_passed'] = 'Yes'
            fig.savefig(os.path.join(path, 'analysis', spline_folder, filename_save), dpi=300, format="png")
            fig.savefig(os.path.join(path, 'analysis', spline_folder, filename_save.replace('.png', '.pdf')), dpi=300, format="pdf")


        else:
            df_ring_data.loc[index,'spline_filter_passed'] = 'No'
            fig.savefig(os.path.join(path, 'analysis', spline_folder_exclude, filename_save), dpi=300, format="png")
            fig.savefig(os.path.join(path, 'analysis', spline_folder_exclude, filename_save.replace('.png', '.pdf')), dpi=300, format="pdf")

    else:
        filename_save = ring['filename_ring'].replace('.hdf5', '_spline.png')

        df_ring_data.loc[index,'spline_filter_passed'] = ''
        df_ring_data.loc[index,'2piradius_merge'] = np.nan
        df_ring_data.loc[index,'spline_length'] = np.nan
        df_ring_data.loc[index,'N_peaks'] = np.nan

        
    
    
    
df_ring_data = df_ring_data.drop(columns=['filename_ring'])



df_ring_data.to_csv(os.path.join(path,"ring_data_unfold_filter"+folder_suffix+".csv"))
# Save dataframe with cell means for easy loading of data for postprocessing
df_ring_data.to_pickle(os.path.join(path,"ring_data_unfold_filter"+folder_suffix+".pkl"))



Processing rings:   0%|          | 0/418 [00:00<?, ?it/s]

In [ ]:
## 